In [1]:
import itertools
import numpy as np
from matplotlib import pyplot
import math

from PySDM import Builder, Formulae
from PySDM.dynamics import Freezing
from PySDM.environments import Box
from PySDM.physics import constants_defaults as const
from PySDM.physics import si
from PySDM.products import IceWaterContent
from PySDM.backends import CPU, GPU

from open_atmos_jupyter_utils import show_plot

/home/slayoo/devel/PySDM/PySDM/backends/__init__.py:33: UserWarning: CUDA library found but cuInit() failed (error code: 999; message: unknown error)
  warnings.warn(


In [2]:
DTS = (1000, 2000, 4000, 8000, 16000, 32000)
MLT = (4, 16, 64, 256, 1024, 4096)
SEEDS = (22, 33, 44, 55, 66, 77)

def error_norm(actual, expected):
    return np.sqrt(np.mean(np.square(actual - expected)))

In [3]:
cases = tuple([{"dt": dt, "N": mlt, "seed": seed} for dt, mlt, seed in itertools.product(DTS, MLT, SEEDS)])
rate = 1e-9 / si.s
immersed_surface_area = 1 * si.m

number_of_real_droplets = 1024*1
total_time = (
    0.2e8  # effectively interpreted here as seconds, i.e. cycle = 1 * si.s
)

# dummy (but must-be-set) values
vol = (
    44  # for sign flip (ice water has negative volumes), value does not matter
)
d_v = 666  # products use conc., dividing there, multiplying here, value does not matter

def hgh(t):
    return np.exp(-0.75 * rate * (t - total_time / 4))

def low(t):
    return np.exp(-1.25 * rate * (t + total_time / 4))


output = {}

products = (IceWaterContent(name="qi"),)

for backend_class in (GPU,):
    backend_key = backend_class.__name__
    output[backend_key] = {}
    for case in cases:
        formulae = Formulae(
            heterogeneous_ice_nucleation_rate="Constant",
            constants={"J_HET": rate / immersed_surface_area},
            seed=case['seed'],
        )
        
        n_sd = int(number_of_real_droplets // case["N"])
        assert n_sd == number_of_real_droplets / case["N"]
        assert total_time // case["dt"] == total_time / case["dt"]

        key = f"{case['dt']}:{n_sd}:{case['seed']}"
        print(key, "...")
        output[backend_key][key] = {"unfrozen_fraction": [], "dt": case["dt"], "N": case["N"]}

        builder = Builder(n_sd=n_sd, backend=backend_class(formulae=formulae))
        env = Box(dt=case["dt"], dv=d_v)
        builder.set_environment(env)
        builder.add_dynamic(Freezing(singular=False))
        attributes = {
            "multiplicity": np.full(n_sd, int(case["N"])),
            "immersed surface area": np.full(n_sd, immersed_surface_area),
            "volume": np.full(n_sd, vol),
        }
        particulator = builder.build(attributes=attributes, products=products)
        env["RH"] = 1.0001
        env["a_w_ice"] = np.nan
        env["T"] = np.nan

        cell_id = 0
        for i in range(int(total_time / case["dt"]) + 1):
            particulator.run(0 if i == 0 else 1)

            ice_mass_per_volume = particulator.products["qi"].get()[cell_id]
            ice_mass = ice_mass_per_volume * d_v
            ice_number = ice_mass / (const.rho_w * vol)
            unfrozen_fraction = 1 - ice_number / number_of_real_droplets
            output[backend_key][key]["unfrozen_fraction"].append(unfrozen_fraction)

1000:256:22 ...


/home/slayoo/devel/PySDM/PySDM/backends/thrust_rtc.py:72: UserWarning: CUDA is not available, using FakeThrustRTC!
  warnings.warn("CUDA is not available, using FakeThrustRTC!")


1000:256:33 ...


/home/slayoo/devel/PySDM/PySDM/backends/thrust_rtc.py:72: UserWarning: CUDA is not available, using FakeThrustRTC!
  warnings.warn("CUDA is not available, using FakeThrustRTC!")


1000:256:44 ...


...

1000:1:66 ...


/home/slayoo/devel/PySDM/PySDM/backends/thrust_rtc.py:72: UserWarning: CUDA is not available, using FakeThrustRTC!
  warnings.warn("CUDA is not available, using FakeThrustRTC!")


1000:1:77 ...


/home/slayoo/devel/PySDM/PySDM/backends/thrust_rtc.py:72: UserWarning: CUDA is not available, using FakeThrustRTC!
  warnings.warn("CUDA is not available, using FakeThrustRTC!")


AssertionError: 

In [ ]:
def _plot_fit(fit_x, fit_y, low, hgh, total_time):
    pyplot.plot(
        fit_x, fit_y, color="black", linestyle="--", label="theory", linewidth=5
    )
    pyplot.plot(
        fit_x, hgh(fit_x), color="black", linestyle=":", label="assert upper bound"
    )
    pyplot.plot(
        fit_x, low(fit_x), color="black", linestyle=":", label="assert lower bound"
    )
    pyplot.legend()
    pyplot.yscale("log")
    pyplot.ylim(fit_y[-1], fit_y[0])
    pyplot.xlim(None, total_time)
    pyplot.xlabel("time [s]")
    pyplot.ylabel("unfrozen fraction")
    pyplot.grid()

In [ ]:
for backend_key in output.keys():
    for key, out in output[backend_key].items():
        fit_x = np.linspace(0, total_time, num=100)
        fit_y = np.exp(-rate * fit_x)

        sim_x = out["dt"] * np.arange(len(out["unfrozen_fraction"]))
        sim_y = out["unfrozen_fraction"]
#         pyplot.step(
#             sim_x,
#             sim_y,
#             label=f"dt={out['dt']:.2g} / N={out['N']}",
#             marker=".",
#            # linewidth=1 + out["N"] // 8,
#         )
        output[backend_key][key]['mse'] = error_norm(
            actual=sim_y[1:],
            expected=np.exp(-rate * sim_x[1:])
        )
#     _plot_fit(fit_x, fit_y, low, hgh, total_time)
#     pyplot.title(f"backand: {backend_key}")
#     show_plot(f"fig_convergence_{backend_key}.pdf")

In [ ]:
data = {}
serr = {}
for backend_key in output.keys():
    data[backend_key] = np.zeros(shape=(len(DTS), len(MLT)))
    serr[backend_key] = np.zeros(shape=(len(DTS), len(MLT)))
    for key in output[backend_key].keys():
        dt, mlt, _ = key.split(':')
        i = DTS.index(float(dt))
        j = MLT.index(number_of_real_droplets // int(mlt))
        data[backend_key][i, j] += output[backend_key][key]['mse']
        serr[backend_key][i, j] += output[backend_key][key]['mse']**2
    data[backend_key] /= len(SEEDS)
    serr[backend_key] /= len(SEEDS)
    serr[backend_key] -= data[backend_key]**2
    serr[backend_key] = np.where(
        serr[backend_key] != 0,
        np.sqrt(serr[backend_key] / len(SEEDS)),
        np.nan
    )

def phi(dts):
    l2dts = np.log2(dts)
    scaled = (l2dts - min(l2dts)) / (max(l2dts) - min(l2dts))
    return scaled * np.pi / 2

def rho(n):
    return np.log2(n)

for backend_key in output.keys():
    theta_array = phi(total_time // np.asarray(DTS))
    r_array = rho(number_of_real_droplets // np.asarray(MLT))

    X, Y = np.meshgrid(theta_array, r_array)
    Z = np.array(list(np.log2(data[backend_key]))).reshape(len(MLT), len(DTS)).T 
    
    rng = [math.floor(np.amin(Z)), math.ceil(np.amax(Z))]
    levels = np.linspace(*rng, rng[-1] - rng[0] + 1)
    
    ax = pyplot.subplot(111, projection='polar')
    cnt = ax.contourf(X, Y, Z, levels, cmap='jet')
    pyplot.contour(X, Y, Z, levels, colors='black')
    
    ax.scatter(X, Y, alpha=.8, s=10)
    legend = pyplot.colorbar(cnt, ax=ax, pad=0.1)
    legend.set_label(r'$log_2(Err_{L2})$', rotation=90)

    ax.set_xlabel(r"$n_{sd}$", labelpad=18)
    ax.set_rlim(min(r_array) - 1, max(r_array))
    ax.set_yticks(r_array)
    ax.set_yticklabels("$2^{" + f"{int(tick):d}" + "}$" for tick in r_array)
    
    ax.annotate(r'$dt$ [s]', xy=(.8, .85), xycoords='axes fraction')
    ax.set_thetalim(min(theta_array), max(theta_array))
    ax.set_thetagrids(theta_array / (np.pi/2) * 90, tuple(f"{dt:d}" for dt in DTS))

    ax.grid(True)
    
    show_plot(f'fig_contours_{backend_key}.pdf')
    
    ax = pyplot.subplot(111)
    for i, dt in enumerate(DTS):
        ax.errorbar(
            x=number_of_real_droplets // np.asarray(MLT),
            y=data[backend_key][i, :],
            yerr=serr[backend_key][i, :],
            label=f"dt={dt} s",
            marker='o',
            capsize=4
        )
    pyplot.xscale('log', base=2)
    pyplot.yscale('log', base=2)
    pyplot.legend()
    pyplot.xlabel("$n_{sd}$")
    pyplot.ylabel(r'$Err_{L2} (bars: std. err.)$')
    pyplot.grid()
    show_plot(f'fig_lines_{backend_key}.pdf')